In [2]:
#importing libraries
import pandas as pd
import requests
import zipfile
import os
import shutil

In [ ]:
# Step 1 : For downloading file
for year in range(1975,2021,5):
    
    # Making the url and filename with year substituion
    url = f"https://static.nhtsa.gov/nhtsa/downloads/FARS/{year}/National/FARS{year}NationalCSV.zip"
    filename = f"FARS{year}NationalCSV.zip"
    
    print(f'Start downloading file for year : {year}')
    
    # Calling requests library/module to download the url
    r = requests.get(url, allow_redirects=True)
    
    # opening a file for writing 
    # 'wb': for writing byte mod
    # 'w' : writing file in string mode
    with open(filename,'wb') as f:
        f.write(r.content)
    
    print(f'Download completed for year :{year}')

In [12]:
# Step 2 : To uncompress the files into year directory

# Use os module to generate/create directory with year
for year in range(1975,2021,5):
    directory = f'{year}'
    # os module has path submodule
    # which has a function exists
    # this function checks if path/directory exists : returns true
    # else it returns false
    if not os.path.exists(directory):
        # in os module, mkdir function creates the 
        # path/directory passed to it
        os.mkdir(directory)

In [13]:
#extracting the files into year directory
for year in range(1975,2021,5):
    filename = f"FARS{year}NationalCSV.zip"
    directory = f'{year}'
    
    # calling a ZipFile function from zipfile module
    # to open a zip file
    with zipfile.ZipFile(filename,'r') as zip_ref:
        # extracting the files into year directory
        zip_ref.extractall(directory)

In [14]:
# Step 3 : copying all accident.csv files into directory accident_all_years
newDir = "accident_all_years"
if not os.path.exists(newDir):
    os.mkdir(newDir)

In [15]:
for year in range(1975,2021,5):
    for file in [ 'ACCIDENT.CSV' ,'accident.csv']:
        oldPath = f'{year}/{file}'
        newPath = f'{newDir}/{year}_accident.csv'
        if os.path.exists(oldPath):
            # print(oldPath)
            # print(newPath)
            shutil.copy(oldPath,newPath)
        else:
            oldPath = f'{year}/FARS{year}NationalCSV/{file}'
            newPath = f'{newDir}/{year}_accident.csv'
            if os.path.exists(oldPath):
                # print(oldPath)
                # print(newPath)
                shutil.copy(oldPath,newPath)            

In [24]:
# initiate variables for totalAccidents and fatalAccidents
totalAccidents = 0
fatalAccidents = 0

#fromthe directory accident_all_years folder iterate over all files from 1975 to 2020
for year in range(1975,2021,5):
    #create a dataframe for each year
    df = pd.read_csv(f'accident_all_years/{year}_accident.csv',encoding='ISO-8859-1')
     
    
    """ print(f"{year} : {df.loc[df.FATALS > 0,'FATALS'].count()} : {df.FATALS.value_counts().get(0)} : {df.shape[0]}")
    if df.FATALS.value_counts().get(0) == None :
        fatalAccidents += (df.shape[0])
    else:
        fatalAccidents += (df.shape[0] - df.FATALS.value_counts().get(0))
        totalAccidents +=  df.shape[0]  """
   
    
    # print(f"{year} : {df.loc[df.FATALS > 0,'FATALS'].count()} : {df.loc[df.FATALS == 0,'FATALS'].count()} : {df.shape[0]}")
    fatalAccidents += (df.shape[0] - df.loc[df.FATALS == 0,'FATALS'].count())
    totalAccidents +=  df.shape[0] 

print(f"Total Accidents : {totalAccidents}")
print(f"Total Fatal Accidents : {fatalAccidents}")

Total Accidents : 376265
Total Fatal Accidents : 376255
